In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!ls
import os
os.chdir("drive/My Drive/Diplomski rad/slow-fast/")
!ls

drive  sample_data
chpt	   ngrok-stable-linux-amd64.zip    rs50		    test.py
chpt2	   ngrok-stable-linux-amd64.zip.1  rs50-but-mob     train.ipynb
fuse.py    __pycache__			   saved.hd5	    Untitled.ipynb
logs	   resnet.py			   slow-only
models.py  rs18				   slow-only.ipynb
ngrok	   rs18-24frames		   stems.py


In [0]:
frames = 32

In [4]:
import tensorflow
from tensorflow.keras import Model, Sequential

from tensorflow.keras.layers import Dense, GlobalMaxPooling1D, Flatten, TimeDistributed, Input, Lambda, Concatenate
from tensorflow.keras.optimizers import Adam
import numpy as np

#from models import SlowFastModel
from stems import InitialStem, FinalStem
from fuse import FuseFastConv
from resnet import ResStage, Bottleneck, res_stage, Basic
tensorflow.keras.backend.set_image_data_format('channels_last')

backbone = 18
data_dim = 224
dropout = 0.2
_RESNET_MODEL_DEPTH = {18:(2,1,1,1), 50: (3, 4, 6, 3), 101: (3, 4, 23, 3)}

_RESNET_STRIDES = [1, 2, 2, 2]

_RESNET_DILATIONS = [1, 1, 1, 1]

_RESNET_T_FUNC = Bottleneck

_CLASSES = 101

_TEMPORAL_KERNEL_SIZES = {"slowfast": [
        [[1], [5]],  # conv1 temporal kernel for slow and fast pathway.
        [[1], [3]],  # res2 temporal kernel for slow and fast pathway.
        [[1], [3]],  # res3 temporal kernel for slow and fast pathway.
        [[3], [3]],  # res4 temporal kernel for slow and fast pathway.
        [[3], [3]],  # res5 temporal kernel for slow and fast pathway.
    ],}

_RESNET_KERNEL_LIMIT = [2, 1, 1, 1]

_POOLS = {
    "slowfast": [[1, 1, 1], [1, 1, 1]],
}



alpha = 8

beta_inv = 8
fusion_channel_ratio = 2
fusion_kernel_size= 5
pool_size = _POOLS["slowfast"]

def prepare_input(alpha):
    def func(x):
        b = tensorflow.identity(x[:,::alpha])
        return b
    return Lambda(func)

data_channels = [3,3]
inp_layer = Input((frames,224,224,3))
p_i = prepare_input(alpha)(inp_layer)

(b1,b2,b3,b4) = _RESNET_MODEL_DEPTH[backbone]
        
        # we will just use config for RESNET, see ResNE(x)T
width_per_group = 64
number_of_groups = 1

dim_inner = width_per_group * number_of_groups

out_ratio = (beta_inv//fusion_channel_ratio)

temp_kernel = _TEMPORAL_KERNEL_SIZES["slowfast"]


d_out = [width_per_group, width_per_group // beta_inv]
k = [temp_kernel[0][0] + [7, 7], temp_kernel[0][1] + [7, 7]]
s = [(1, 2, 2), (1, 2, 2)]
pad = [
[temp_kernel[0][0][0] // 2, 3, 3],
[temp_kernel[0][1][0] // 2, 3, 3]
]

i_s1 = InitialStem(data_channels[0], d_out[0], k[0], s[0], pad[0] )(p_i)
#i_s2 = InitialStem(data_channels[1], d_out[1], k[1], s[1], pad[1] )(inp_layer)

 #r1 = SlowFastModel(50, alpha, 8, 2, 5, 32, 0.1, 224)([k,inp_layer])
# i_s2 is fast!
#fused_slow_1 = FuseFastConv(width_per_group//beta_inv,fusion_channel_ratio,fusion_kernel_size, alpha)([i_s1,i_s2])

stem2_slow = res_stage(dim_in=width_per_group+width_per_group//out_ratio,
            dim_out=width_per_group*4,
            dim_inner=dim_inner,
            temp_kernel_size=temp_kernel[1][0],
            stride=_RESNET_STRIDES[0],
            number_of_blocks=b1,
            num_groups=number_of_groups,
            num_block_temp_kernel=_RESNET_KERNEL_LIMIT[0],
            t_func=_RESNET_T_FUNC,
            dilation=_RESNET_DILATIONS[0],
)(i_s1)
"""stem2_fast = res_stage(
    dim_in = width_per_group//beta_inv,
    dim_out = width_per_group*4//beta_inv,
    dim_inner = dim_inner//beta_inv,
    temp_kernel_size=temp_kernel[1][1],
    stride=_RESNET_STRIDES[0],
    number_of_blocks=b1,
    num_groups=number_of_groups,
    num_block_temp_kernel=_RESNET_KERNEL_LIMIT[0],
    t_func=_RESNET_T_FUNC,
    dilation=_RESNET_DILATIONS[0]
)(i_s2)
"""
#fused_slow_2 = FuseFastConv(width_per_group*4//beta_inv,fusion_channel_ratio,fusion_kernel_size, alpha)([stem2_slow,stem2_fast])

pool1_slow = tensorflow.keras.layers.MaxPooling3D(pool_size=pool_size[0],strides=pool_size[0])(stem2_slow)
#pool1_fast = tensorflow.keras.layers.MaxPooling3D(pool_size=pool_size[1],strides=pool_size[1])(stem2_fast)

stem3_slow = res_stage(dim_in=width_per_group*4+width_per_group*4//out_ratio,
            dim_out=width_per_group*8,
            dim_inner=dim_inner*2,
            temp_kernel_size=temp_kernel[2][0],
            stride=_RESNET_STRIDES[1],
            number_of_blocks=b2,
            num_groups=number_of_groups,
            num_block_temp_kernel=_RESNET_KERNEL_LIMIT[1],
            t_func=_RESNET_T_FUNC,
            dilation=_RESNET_DILATIONS[1],
)(pool1_slow)
"""stem3_fast = res_stage(
    dim_in = width_per_group*4//beta_inv,
    dim_out = width_per_group*8//beta_inv,
    dim_inner = dim_inner*2//beta_inv,
    temp_kernel_size=temp_kernel[2][1],
    stride=_RESNET_STRIDES[1],
    number_of_blocks=b2,
    num_groups=number_of_groups,
    num_block_temp_kernel=_RESNET_KERNEL_LIMIT[1],
    t_func=_RESNET_T_FUNC,
    dilation=_RESNET_DILATIONS[1]
)(pool1_fast)
"""
#fused_slow_3 = FuseFastConv(width_per_group*8//beta_inv,fusion_channel_ratio,fusion_kernel_size, alpha)([stem3_slow,stem3_fast])

stem4_slow = res_stage(dim_in=width_per_group*8+width_per_group*8//out_ratio,
            dim_out=width_per_group*16,
            dim_inner=dim_inner*4,
            temp_kernel_size=temp_kernel[3][0],
            stride=_RESNET_STRIDES[2],
            number_of_blocks=b3,
            num_groups=number_of_groups,
            num_block_temp_kernel=_RESNET_KERNEL_LIMIT[2],
            t_func=_RESNET_T_FUNC,
            dilation=_RESNET_DILATIONS[2],
)(stem3_slow)
"""stem4_fast = res_stage(
    dim_in = width_per_group*8//beta_inv,
    dim_out = width_per_group*16//beta_inv,
    dim_inner = dim_inner*4//beta_inv,
    temp_kernel_size=temp_kernel[3][1],
    stride=_RESNET_STRIDES[2],
    number_of_blocks=b3,
    num_groups=number_of_groups,
    num_block_temp_kernel=_RESNET_KERNEL_LIMIT[2],
    t_func=_RESNET_T_FUNC,
    dilation=_RESNET_DILATIONS[2]
)(stem3_fast)
"""
#fused_slow_4 = FuseFastConv(width_per_group*16//beta_inv,fusion_channel_ratio,fusion_kernel_size, alpha)([stem4_slow,stem4_fast])

stem5_slow = res_stage(dim_in=width_per_group*16+width_per_group*16//out_ratio,
            dim_out=width_per_group*32,
            dim_inner=dim_inner*8,
            temp_kernel_size=temp_kernel[4][0],
            stride=_RESNET_STRIDES[3],
            number_of_blocks=b4,
            num_groups=number_of_groups,
            num_block_temp_kernel=_RESNET_KERNEL_LIMIT[3],
            t_func=_RESNET_T_FUNC,
            dilation=_RESNET_DILATIONS[3],
)(stem4_slow)
"""stem5_fast = res_stage(
    dim_in = width_per_group*16//beta_inv,
    dim_out = width_per_group*32//beta_inv,
    dim_inner = dim_inner*8//beta_inv,
    temp_kernel_size=temp_kernel[4][1],
    stride=_RESNET_STRIDES[3],
    number_of_blocks=b4,
    num_groups=number_of_groups,
    num_block_temp_kernel=_RESNET_KERNEL_LIMIT[3],
    t_func=_RESNET_T_FUNC,
    dilation=_RESNET_DILATIONS[3]
)(stem4_fast)
"""
stem_final = FinalStem(dim_in=[width_per_group*32],
        classes=_CLASSES,
        pool=[[frames//alpha//pool_size[0][0], data_dim//32//pool_size[0][1], data_dim//32//pool_size[0][2]]],
        dropout=dropout,
        pathways=1)([stem5_slow,stem4_slow])

model = Model(inp_layer, stem_final)


model.summary()


TEMPORAL KERNEL SIZE [1, 1]
resnet_block dim_in:80 dim_out:256
dimout-->256
BOTTLENECK []
RESBLOCK []
resnet_block_1 dim_in:256 dim_out:256
BOTTLENECK []
RESBLOCK []
TEMPORAL KERNEL SIZE [1]
resnet_block_2 dim_in:320 dim_out:512
dimout-->512
BOTTLENECK []
RESBLOCK []
TEMPORAL KERNEL SIZE [3]
resnet_block_3 dim_in:640 dim_out:1024
dimout-->1024
BOTTLENECK []
RESBLOCK []
TEMPORAL KERNEL SIZE [3]
resnet_block_4 dim_in:1280 dim_out:2048
dimout-->2048
BOTTLENECK []
RESBLOCK []
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 224, 224 0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 4, 224, 224,  0           input_1[0][0]                    
_____________

In [5]:
data = np.array([np.random.rand(frames,data_dim,data_dim,3),np.random.rand(frames,data_dim,data_dim,3)])
print(model.predict(data).shape)

(2, 101)


In [0]:
os.chdir("../data/")

In [7]:
from new_generator import FrameGenerator
import tensorflow
import glob
import os
import pickle
import numpy

os.chdir("./Videos/")
classes = [i.split(os.path.sep)[1] for i in glob.glob('train/*')]
classes.sort()
print(len(classes))
SIZE = (224, 224)
CHANNELS = 3
NBFRAME = frames
BS = 4
EPOCHS = 20


glob_pattern='train/{classname}/*.avi'

data_aug = tensorflow.keras.preprocessing.image.ImageDataGenerator(
    zoom_range=.2,
    horizontal_flip=True,
    rotation_range=6,
    width_shift_range=.15,
    height_shift_range=.15)

t_list=None
v_list=None

with open("trainlist2","rb") as fp:
    t_list=pickle.load(fp)

with open("validlist2","rb") as fp:
    v_list=pickle.load(fp)

train = FrameGenerator(
    BS,
    NBFRAME,
    classes, 
    SIZE,
    None,
    CHANNELS,
    glob_pattern,
    data_aug,
    typ="train",
    prepared_data=t_list
)

valid = FrameGenerator(
    BS,
    NBFRAME,
    classes, 
    SIZE,
    None,
    CHANNELS,
    glob_pattern,
    None,
    typ="valid",
    prepared_data=v_list
)

101
Loaded 7185 files for train generator.
Loaded 2352 files for valid generator.


Using TensorFlow backend.


In [9]:
import tensorflow
from tensorflow.keras.models import load_model

model = load_model("../../slow-fast/slow-only/")

ValueError: ignored

In [8]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("../../slow-fast/slow-only/", monitor='val_loss', verbose=1, save_best_only=True, mode='auto', save_freq="epoch")

opt = Adam(0.001)#0.0001
model.compile(loss=tensorflow.keras.losses.categorical_crossentropy, optimizer=opt, metrics=["accuracy"])
print(len(valid)//valid.batch_size)
model.fit(
    train,
    steps_per_epoch=len(train),
    validation_data=valid,
    validation_steps=len(valid),
    verbose=1,
    epochs=EPOCHS,
    callbacks=[checkpoint],
)

147
Epoch 1/20
1796/1796 [==============================] - ETA: 0s - loss: 4.6362 - accuracy: 0.0323Invalid frame count for video train/PushUps/v_PushUps_g16_c04.avi . Requested: 32 Obtained: 29
No other option for train/PushUps/v_PushUps_g16_c04.avi
Could not get frames for train/PushUps/v_PushUps_g16_c04.avi

Epoch 00001: val_loss improved from inf to 6.45502, saving model to ../../slow-fast/slow-only/
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ../../slow-fast/slow-only/assets
1796/1796 [==============================] - 19945s 11s/step - loss: 4.6362 - accuracy: 0.0323 - val_loss: 6.4550 - val_accuracy: 0.0617
Epoch 2/20
1447/1796 [=======================>......] - ETA: 59:48 - loss: 4.2099 - accuracy: 0.0662

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-2cadfd5e4a11>", line 15, in <module>
    callbacks=[checkpoint],
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py", line 66, in _method_wrapper
    return method(self, *args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py", line 848, in fit
    tmp_logs = train_function(iterator)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/eager/def_function.py", line 580, in __call__
    result = self._call(*args, **kwds)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/eager/def_function.py", line 611, in _call
    return self._stateless_fn(*args, **kwds)  # pylint: disable=not-callable
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/eager/

UnknownError: ignored